In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Chrome_options = Options()
Chrome_options.add_argument("--verbose")
Chrome_options.add_argument('--no-sandbox')
Chrome_options.add_argument("--headless=new")
Chrome_options.add_argument("--disable-gpu")
Chrome_options.add_argument("--windows-size=1920,1200")
Chrome_options.add_argument("--disable-dev-shm-usage")
Chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=Chrome_options)

In [2]:
def safe_find_element(driver, by, value):
    try:
        return driver.find_element(by, value)
    except NoSuchElementException:
        return None

def news_scraping(news_url, driver):
    # 언론사
    press_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[2]')
    press = press_element.get_attribute('title') if press_element else ""

    # 기사 제목
    title_element = safe_find_element(driver, By.ID, 'title_area')
    title = title_element.text if title_element else ""

    # 발행일자
    date_time_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span')
    date_time = date_time_element.text if date_time_element else ""

    # 기자
    repoter_element = safe_find_element(driver, By.XPATH, '//*[@id="JOURNALIST_CARD_LIST"]/div[1]/div/div[1]/div/div/div[1]/a[2]/span/span/em')
    repoter = repoter_element.text if repoter_element else ""

    # 기사 본문
    article_element = safe_find_element(driver, By.ID, 'dic_area')
    article = article_element.text.replace("\n", "").replace("\t", "") if article_element else ""

    # 기사 반응: 쏠쏠정보
    useful_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[1]/a/span[2]')
    useful = useful_element.text if useful_element else ""

    # 기사 반응: 흥미진진
    wow_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[2]/a/span[2]')
    wow = wow_element.text if wow_element else ""

    # 기사 반응: 공감백배
    touched_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[3]/a/span[2]')
    touched = touched_element.text if touched_element else ""

    # 기사 반응: 분석탁월
    analytical_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[4]/a/span[2]')
    analytical = analytical_element.text if analytical_element else ""

    # 기사 반응: 후속강추
    recommend_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[5]/a/span[2]')
    recommend = recommend_element.text if recommend_element else ""

    print("뉴스:", [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url])

    return [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url]

def scraping(list_url):
    driver.implicitly_wait(3)

    news_idx = 1
    news_df = pd.DataFrame(columns = ("Title", "Press", "DateTime", "Repoter", "Article", "Useful", "Wow", "Touched", "Analytical", "Recommend", "URL"))

    for url in list_url:
        driver.get(url)
        news_df.loc[news_idx] = news_scraping(url, driver)
        news_idx += 1

    driver.close()

    return news_df

In [3]:
def make_pg_num(num):
    """Calculate the page number in the format required by the website."""
    return num if num == 1 else num+9*(num-1)

def create_url(search, page_num):
    """Create a URL with the search term and page number."""
    return f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={page_num}"

def make_urls(search, start_pg, end_pg):
    """Generate the URLs for the range of pages."""
    return [create_url(search, make_pg_num(i)) for i in range(start_pg, end_pg+1)]

def input_with_validation(prompt):
    """Ask for input with the given prompt, repeating until a valid integer is provided."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input, please enter an integer.")

def main():
    search = input("검색 키워드를 입력해주세요: ")

    start_pg = input_with_validation("\n크롤링 시작 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 시작 페이지: {start_pg}페이지")

    end_pg = input_with_validation("\n크롤링 종료 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 종료 페이지: {end_pg}페이지")

    return make_urls(search, start_pg, end_pg)

if __name__ == "__main__":
    search_urls = main()
    print("생성된 URL: ", search_urls)


크롤링 시작 페이지: 1페이지

크롤링 종료 페이지: 2페이지
생성된 URL:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경기도북부특별자치도&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경기도북부특별자치도&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11']


In [5]:
# Initialize the list to store the links
list_url = []

# Iterate over the URLs
for url in search_urls:
    # Send GET request to the web page
    response = requests.get(url)

    # If the request is successful, extract the HTML content and create a BeautifulSoup object
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("a.info, a.sub_txt")

        # Filter and save the links with "naver.com" in their address
        for link in links:
            href = link.get("href")
            if "naver.com" in href:
                list_url.append(href)
    else:
        print("The request failed.")

    # Sleep for 1 second
    time.sleep(2)

In [6]:
scraping(list_url)

뉴스: ["경기도의회 '경기북부특별자치도 특위' 의장 직권상정하나", '뉴시스', '2023.06.21. 오후 4:48', '이병희 기자', '의회운영위 심의 불발…27일 \'원포인트\' 회의 예정염종현 의장 "처리 안되면 본회의 직권상정" 예고경기도의회 광교신청사. (사진=경기도의회 제공) *재판매 및 DB 금지[수원=뉴시스] 이병희 기자 = 수개월째 경기도의회 상임위원회에서 계류 중인 \'경기북부특별자치도 설치 특별위원회 구성 결의안\'을 염종현 경기도의회 의장이 본회의 직권상정할지 관심이 모아지고 있다.소관 상임위원회인 의회운영위원회가 심의를 미루면서 처리 여부는 여전히 안갯속인 상태다.경기도의회 의회운영위원회는 21일 제369회 정례회 제1차 회의를 열었지만 \'경기북부특별자치도 설치 특별위원회 구성 결의안\'을 상정하지 않기로 했다. 해당 결의안은 본회의 전날인 오는 27일 특위 결의안 처리를 위한 \'원포인트\' 회의를 열어 심의·의결할 예정이다.이 특별위원회는 경기북부특별자치도 설치를 위한 각종 계획의 수립과 도 차원의 관련 추진 정책을 지원하기 위해 추진된다.염종현(민주당·부천1) 의장을 비롯해 전체 의원(156명)의 91%인 142명이 공동 발의자로 이름을 올린 이 결의안은 당초 지난 4월 제368회 임시회에 회부됐지만, 여야 의견대립으로 의회운영위원회가 열리지 않으면서 지금껏 계류 중이다.이에 염 의장은 지난 13일 개회사에서 "의원의 91%에 달하는 142명이 서명한 안건이 구성은 물론 활동조차도 하지 못하고 있다"면서 "상황을 주도하고, 논의의 중심에 있어야 할 경기도의회가 언제까지 이렇게 손을 놓고 있어야 하나"라고 지적했다.그러면서 "여야 합의를 통해 특별위원회 구성이 원만하게 타결되기를 기다려 왔지만 이제 더는 지체할 수 없다"면서 결의안이 상임위원회에서 처리되지 않을 경우 의장 직권으로 본회의 상정을 검토하고 있다고 밝혔다.\'경기도의회 회의규칙\' 제26조 2항 17호에 따르면 본회의 중 의장의 제의 의안은 위원회에 회부하지 않고

,Title,Press,DateTime,Repoter,Article,Useful,Wow,Touched,Analytical,Recommend,URL
1,경기도의회 '경기북부특별자치도 특위' 의장 직권상정하나,뉴시스,2023.06.21. 오후 4:48,이병희 기자,"의회운영위 심의 불발…27일 '원포인트' 회의 예정염종현 의장 ""처리 안되면 본회의...",0,0,0,0,0,https://n.news.naver.com/mnews/article/003/001...
2,의장 엄포에도 경기도의회 '북부특별자치도 특위' 안건 처리지연,연합뉴스,2023.06.21. 오후 4:53,최찬흥 기자,다른 특위 구성안 일괄처리 놓고 이견…상임위 불발 시 28일 의장 직권상정(수원=연...,0,0,0,0,0,https://n.news.naver.com/mnews/article/001/001...
3,"오후석 경기도 부지사, “경기북부특별자치도 설치, 가장 필요한 것은 시민의 자발적 참여”",헤럴드경제,2023.06.22. 오전 7:17,박준환 기자,‘경기북부특별자치도 시·군별 토론회’ 포천시에서 시작[헤럴드경제=박준환 기자]경기북...,0,0,0,0,0,https://n.news.naver.com/mnews/article/016/000...
4,경기북부특별자치도 설치 공론화 위한 첫 토론회 포천서 열려,이데일리,2023.06.21. 오후 3:29,정재훈 기자,경기북부청년단체 '규제철폐 우선' 기자회견도[포천=이데일리 정재훈 기자] 김동연 경...,0,0,0,0,0,https://n.news.naver.com/mnews/article/018/000...
5,"이동환 고양특례시장 ""경기북부 자립하려면…수도권 규제 완화 필요하다""",한국경제,2023.06.21. 오후 6:03,강준완 기자,"""각종 제약에 성장 기반 못 다져기업 유치 위해 정비법 풀어야북부 10개 시·군 한...",1,0,0,0,0,https://n.news.naver.com/mnews/article/015/000...
6,‘경기북부특별자치도 설치’ 공감대 형성…포천서 토론회 열려,국민일보,2023.06.21. 오후 4:27,박재구 기자,포천시 시작해 경기북부 10개 시·군 순차 진행경기북부 시민단체 ‘수정법 개정 우선...,1,0,0,0,0,https://n.news.naver.com/mnews/article/005/000...
7,'시민공감대 형성'…경기북부특별자치도 설치 첫 토론회 개최,뉴시스,2023.06.21. 오후 4:42,김도희 기자,"오후석 행정2부지사 ""'시민의 자발성'이 가장 중요""경기북부의 문제와 잠재력, 비전...",0,0,0,0,0,https://n.news.naver.com/mnews/article/003/001...
8,"오후석 경기도 부지사, “경기북부특별자치도 출범, 도민들의 열망 크다”",헤럴드경제,2023.06.20. 오전 8:00,박준환 기자,고양특례시 경제인 연합회 주관 제98회 고양포럼에서 역설[헤럴드경제=박준환 기자]오...,0,0,0,0,0,https://n.news.naver.com/mnews/article/016/000...
9,"오후석 경기도 행정2부지사, 고양포럼서 경기북부특별자치도 필요성 강조",데일리안,2023.06.20. 오전 9:21,오명근 기자,[데일리안 = 오명근 기자] 오후석 경기도 행정2부지사가 경기북부지역 경제 기업인을...,0,0,0,0,0,https://n.news.naver.com/mnews/article/119/000...
10,"경기도, ‘북부특별자치도 설치 시·군 순회 토론회’",KBS,2023.06.16. 오전 9:58,임명규 기자,경기도는 오는 21일 포천시부터 다음 달까지 경기북부 10개 시·군 대상 ‘경기북부...,0,0,0,0,0,https://n.news.naver.com/mnews/article/056/001...
